FINAL PROJECT: BEHAVIORIAL CLONING WOOHOO

Collaborators:
- Nicole Wong
- Audrey Li
- Elaine Pham

TODOS:
- Audrey
    1. create yellow and blue colored blocks
- Nicole
    1. reference github code to build model
- Elaine
    1. create random scene generation using yellow and blue blocks

This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/clutter.html).  I recommend having both windows open, side-by-side!

# Working Simulation

Features:

1. Simulation contains foam brick, iiwa, and floor

undefined. Commands iiwa using teleop 

undefined. Manually prints joint angles and corresponding end effector translation

In [ ]:
import logging
import numpy as np
from pydrake.geometry import MeshcatVisualizer, StartMeshcat
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder

from manipulation import running_as_notebook
from manipulation.meshcat_utils import MeshcatPoseSliders, WsgButton
from manipulation.scenarios import (AddIiwaDifferentialIK,
                                    MakeManipulationStation)

from pydrake.all import  (Box, SpatialInertia, UnitInertia, CoulombFriction)

# Start the visualizer.
meshcat = StartMeshcat()

def MakeTeleopAndSimulate():
    model_directives = """
    directives:
    - add_directives:
        file: package://manipulation/iiwa_and_wsg.dmd.yaml
    - add_model:
        name: foam_brick
        file: package://drake/examples/manipulation_station/models/061_foam_brick.sdf
        default_free_body_pose:
            base_link:
                translation: [0, -0.6, 0]
    - add_model:
        name: floor
        file: package://manipulation/floor.sdf
    - add_weld:
        parent: world
        child: floor::box
        X_PC:
            translation: [0, 0, 0]
    """
    builder = DiagramBuilder()

    time_step = 0.001
    station = builder.AddSystem(
        MakeManipulationStation(model_directives, time_step=time_step, 
        # prefinalize_callback=AddBox))
        ))
    plant_robot = station.GetSubsystemByName("plant")
    iiwa = plant_robot.GetModelInstanceByName("iiwa")

    controller_plant = station.GetSubsystemByName(
        "iiwa_controller").get_multibody_plant_for_control()

    # Add a meshcat visualizer.
    meshcat.ResetRenderMode()
    meshcat.DeleteAddedControls()
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, station.GetOutputPort("query_object"), meshcat)

    # Set up differential inverse kinematics.
    differential_ik = AddIiwaDifferentialIK(
        builder,
        controller_plant,
        frame=controller_plant.GetFrameByName("iiwa_link_7"))
    builder.Connect(differential_ik.get_output_port(),
                    station.GetInputPort("iiwa_position"))
    builder.Connect(station.GetOutputPort("iiwa_state_estimated"),
                    differential_ik.GetInputPort("robot_state"))

    # Set up teleop widgets.
    teleop = builder.AddSystem(
        MeshcatPoseSliders(
            meshcat,
            min_range=MeshcatPoseSliders.MinRange(roll=0,
                                                    pitch=-0.5,
                                                    yaw=-np.pi,
                                                    x=-0.6,
                                                    y=-0.8,
                                                    z=0.0),
            max_range=MeshcatPoseSliders.MaxRange(roll=2 * np.pi,
                                                    pitch=np.pi,
                                                    yaw=np.pi,
                                                    x=0.8,
                                                    y=0.3,
                                                    z=1.1),
            body_index=plant_robot.GetBodyByName("iiwa_link_7").index()))
    builder.Connect(teleop.get_output_port(0),
                    differential_ik.get_input_port(0))
    builder.Connect(station.GetOutputPort("body_poses"),
                    teleop.GetInputPort("body_poses"))
    wsg_teleop = builder.AddSystem(WsgButton(meshcat))
    builder.Connect(wsg_teleop.get_output_port(0),
                    station.GetInputPort("wsg_position"))

    diagram = builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    # context = diagram.CreateDefaultContext()
    # default_context = diagram.CreateDefaultContext()
    # plant_context = plant_robot.GetMyContextFromRoot(default_context)
    plant_context = plant_robot.GetMyContextFromRoot(context)
    station_context = station.GetMyContextFromRoot(context)

    if running_as_notebook:  # Then we're not just running as a test on CI.
        simulator.set_target_realtime_rate(1.0)
        meshcat.AddButton("Stop Simulation", "Escape")
        print("Press Escape to stop the simulation")
        while meshcat.GetButtonClicks("Stop Simulation") < 1:
            simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
            # joint angles
            print(station.GetOutputPort("iiwa_position_measured").Eval(station_context))
            gripper_ind = plant_robot.GetBodyIndices(iiwa)[-1]
            gripper = plant_robot.get_body(gripper_ind)
            pose = gripper.EvalPoseInWorld(plant_context).translation()
            angles = plant_robot.GetPositions(plant_context, iiwa)
            print('end effector ', pose)
            print('iiwa joint angles', angles)
        meshcat.DeleteButton("Stop Simulation")
    else:
        simulator.AdvanceTo(10)

MakeTeleopAndSimulate()

Keyboard Controls:
roll : KeyQ / KeyE
pitch : KeyW / KeyS
yaw : KeyA / KeyD
x : KeyJ / KeyL
y : KeyI / KeyK
z : KeyO / KeyU
Press Space to open/close the gripper
Press Escape to stop the simulation
[-0.92340549  0.11064735 -0.73952856 -1.23352896  0.0972912   1.58627993
 -0.07629873]
end effector  [ 4.31745715e-09 -4.40000008e-01  7.60000001e-01]
iiwa joint angles [-0.92340549  0.11064735 -0.73952856 -1.23352896  0.0972912   1.58627993
 -0.07629873]
[-0.92340665  0.11064725 -0.73952719 -1.23352895  0.09729099  1.58627992
 -0.07629857]
end effector  [ 2.35725554e-09 -4.40000015e-01  7.59999995e-01]
iiwa joint angles [-0.92340665  0.11064725 -0.73952719 -1.23352895  0.09729099  1.58627992
 -0.07629857]
[-0.92340664  0.11064725 -0.7395272  -1.23352895  0.09729099  1.58627992
 -0.07629858]
end effector  [ 2.31048646e-09 -4.40000014e-01  7.59999995e-01]
iiwa joint angles [-0.92340664  0.11064725 -0.7395272  -1.23352895  0.09729099  1.58627992
 -0.07629858]
[-0.92340662  0.11064725 -0.739527

## WIP Automating End Effector XYZ and Joint Angles Data Collection

TODO: idk how to leafsystems and periodic events

Pseudo Code:

1. Have a nominal position to reset the robot (follow up question, will the robot ever break getting back to nominal position --> ta says check russ's clutter example for how he resets grip)

undefined. Sample random (x, y, z) end effector positions. Set roll, pitch, yaw to (3.14, 0, 1.58) and do not change. Make sure they are within the ranges ( -.25 <== x <== .3 ) , ( -.4 <== y <== .60 ) , ( table height <== z <== .78) which is the area right in front of the iiwa.

undefined. Do Diff IK in order to calculate the joint angle configurations given end effector (there can be more than one join configuration for an end effector position so is this a good check? ta said that doing diff ik is good in that it diversifies and covers more planar space. will follow up on piazza how using diff ik does this. )

undefined. After every set of (x,y,z) end effector positions, reset to the nominal position to avoid the robot getting stuck in bad positions.

In [ ]:
import logging
import numpy as np
from pydrake.geometry import MeshcatVisualizer, StartMeshcat
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder

from manipulation import running_as_notebook
from manipulation.meshcat_utils import MeshcatPoseSliders, WsgButton
from manipulation.scenarios import (AddIiwaDifferentialIK,
                                    MakeManipulationStation)

from pydrake.all import  (Box, SpatialInertia, UnitInertia, CoulombFriction)

# # Start the visualizer.
meshcat = StartMeshcat()

def RandGripperPose():
    min_x, max_x = -.25, .3
    min_y, max_y = -.4, .6
    min_z, max_z = 0, .78
    rand_xyz = np.array([np.random.uniform(min_x,max_x), 
            np.random.uniform(min_y,max_y), 
            np.random.uniform(min_z,max_z)])
    # desired end effector xyz position
    X_WG_des = RigidTransform(RotationMatrix(), rand_xyz)
    return X_WG_des

def MakeTeleopAndSimulate():
    model_directives = """
    directives:
    - add_directives:
        file: package://manipulation/iiwa_and_wsg.dmd.yaml
    - add_model:
        name: foam_brick
        file: package://drake/examples/manipulation_station/models/061_foam_brick.sdf
        default_free_body_pose:
            base_link:
                translation: [0, -0.6, 0]
    - add_model:
        name: floor
        file: package://manipulation/floor.sdf
    - add_weld:
        parent: world
        child: floor::box
        X_PC:
            translation: [0, 0, 0]
    """
    builder = DiagramBuilder()

    time_step = 0.001
    station = builder.AddSystem(
        MakeManipulationStation(model_directives, time_step=time_step, 
        # prefinalize_callback=AddBox))
        ))
    plant_robot = station.GetSubsystemByName("plant")
    iiwa = plant_robot.GetModelInstanceByName("iiwa")

    controller_plant = station.GetSubsystemByName(
        "iiwa_controller").get_multibody_plant_for_control()

    # Add a meshcat visualizer.
    meshcat.ResetRenderMode()
    meshcat.DeleteAddedControls()
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, station.GetOutputPort("query_object"), meshcat)

    # Set up differential inverse kinematics.
    differential_ik = AddIiwaDifferentialIK(
        builder,
        controller_plant,
        frame=controller_plant.GetFrameByName("iiwa_link_7"))
    builder.Connect(differential_ik.get_output_port(),
                    station.GetInputPort("iiwa_position"))
    builder.Connect(station.GetOutputPort("iiwa_state_estimated"),
                    differential_ik.GetInputPort("robot_state"))

    diagram = builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    plant_context = plant_robot.GetMyContextFromRoot(context)
    station_context = station.GetMyContextFromRoot(context)

    if running_as_notebook:  # Then we're not just running as a test on CI.
        simulator.set_target_realtime_rate(1.0)
        meshcat.AddButton("Stop Simulation", "Escape")
        print("Press Escape to stop the simulation")
        while meshcat.GetButtonClicks("Stop Simulation") < 1:
            simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
            # sample random end effector positions
            planner.getoutputport().Eval(context)
        meshcat.DeleteButton("Stop Simulation")
    else:
        for t in range(1000):
            simulator.AdvanceTo(t*0.01)
            X_WG_des = RandGripperPose()
            angles = differentIK.FixValue()
            plant.getpositions
            
           
        simulator.AdvanceTo(1)

from pydrake.manipulation.planner import (
    DifferentialInverseKinematicsIntegrator,
    DifferentialInverseKinematicsParameters)

def AddIiwaDifferentialIK(builder, plant, frame=None):
    params = DifferentialInverseKinematicsParameters(plant.num_positions(),
                                                     plant.num_velocities())
    time_step = plant.time_step()
    q0 = plant.GetPositions(plant.CreateDefaultContext())
    params.set_nominal_joint_position(q0)
    params.set_end_effector_angular_speed_limit(2)
    params.set_end_effector_translational_velocity_limits([-2, -2, -2],
                                                          [2, 2, 2])
    if plant.num_positions() == 3:  # planar iiwa
        iiwa14_velocity_limits = np.array([1.4, 1.3, 2.3])
        params.set_joint_velocity_limits(
            (-iiwa14_velocity_limits, iiwa14_velocity_limits))
        # These constants are in body frame
        assert (
            frame.name() == "iiwa_link_7"
        ), "Still need to generalize the remaining planar diff IK params for different frames"  # noqa
        params.set_end_effector_velocity_flag(
            [True, False, False, True, False, True])
    else:
        iiwa14_velocity_limits = np.array([1.4, 1.4, 1.7, 1.3, 2.2, 2.3, 2.3])
        params.set_joint_velocity_limits(
            (-iiwa14_velocity_limits, iiwa14_velocity_limits))
        params.set_joint_centering_gain(10 * np.eye(7))
    if frame is None:
        frame = plant.GetFrameByName("body")
    differential_ik = builder.AddSystem(
        DifferentialInverseKinematicsIntegrator(
            plant,
            frame,
            time_step,
            params,
            log_only_when_result_state_changes=True))
    return differential_ik

params = DifferentialInverseKinematicsParameters(plant.num_positions(),
                                                plant.num_velocities())
result = DoDifferentialInverseKinematics(self._plant,
                                            self._plant_context,
                                            V_WE_desired, self._frame_E,
                                            self._diff_ik_params)
if (result.status !=
        DifferentialInverseKinematicsStatus.kNoSolutionFound):
    discrete_state.set_value(
        0, q + self._time_step * result.joint_velocities)

builder.Connect(teleop.get_output_port(0),
                    differential_ik.get_input_port(0))

MakeTeleopAndSimulate()

INFO:drake:Meshcat listening for connections at https://a569df8b-a69d-4307-8cbd-4f24b608e3b4.deepnoteproject.com/7003/


NameError: name 'AbstractValue' is not defined

# WIP Commanding Robot with NN

In [32]:
test_inputs = [RigidTransform(
  R=RotationMatrix([
    [-2.3403549959129465e-06, 0.05032888698822864, -0.9987326985380272],
    [0.9999999999051579, -1.3437349077360507e-05, -3.0204696645260106e-06],
    [-1.3572336781715093e-05, -0.9987326984503733, -0.05032888695200715],
  ]),
  p=[0.4686663152509269, -1.0643346551004039e-05, 0.6012858601360346],
), RigidTransform(
  R=RotationMatrix([
    [0.0, 0.0, -1.0],
    [1.0, 0.0, 0.0],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.55, 0.0, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [0.6427876096865393, 0.0, -0.766044443118978],
    [0.766044443118978, 0.0, 0.6427876096865393],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.5266044443118978, -0.06427876096865393, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [0.984807753012208, 0.0, -0.17364817766693041],
    [0.17364817766693041, 0.0, 0.984807753012208],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.46736481776669303, -0.0984807753012208, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [0.8660254037844387, 0.0, 0.4999999999999998],
    [-0.4999999999999998, 0.0, 0.8660254037844387],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.4, -0.08660254037844388, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [0.3420201433256689, 0.0, 0.9396926207859083],
    [-0.9396926207859083, 0.0, 0.3420201433256689],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.35603073792140916, -0.03420201433256689, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [-0.34202014332566866, 0.0, 0.9396926207859084],
    [-0.9396926207859084, 0.0, -0.34202014332566866],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.35603073792140916, 0.034202014332566866, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [-0.8660254037844384, 0.0, 0.5000000000000004],
    [-0.5000000000000004, 0.0, -0.8660254037844384],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.39999999999999997, 0.08660254037844384, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [-0.9848077530122081, 0.0, -0.17364817766692997],
    [0.17364817766692997, 0.0, -0.9848077530122081],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.46736481776669303, 0.09848077530122082, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [-0.6427876096865396, 0.0, -0.7660444431189778],
    [0.7660444431189778, 0.0, -0.6427876096865396],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.5266044443118978, 0.06427876096865397, 0.4],
), RigidTransform(
  R=RotationMatrix([
    [-2.4492935982947064e-16, 0.0, -1.0],
    [1.0, 0.0, -2.4492935982947064e-16],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.55, 2.4492935982947065e-17, 0.4],
),
  RigidTransform(
  R=RotationMatrix([
    [-2.4492935982947064e-16, 0.0, -1.0],
    [1.0, 0.0, -2.4492935982947064e-16],
    [0.0, -1.0, 0.0],
  ]),
  p=[0.55, 2.4492935982947065e-17, 0.4],
)]


In [43]:
import logging
import numpy as np
from pydrake.geometry import MeshcatVisualizer, StartMeshcat
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder

from manipulation import running_as_notebook
from manipulation.meshcat_utils import MeshcatPoseSliders, WsgButton
from manipulation.scenarios import (AddIiwaDifferentialIK,
                                    MakeManipulationStation, AbstractValue)

from pydrake.all import (Box, SpatialInertia, UnitInertia, CoulombFriction, 
                         LeafSystem, ConstantVectorSource)

import torch
from torch import nn

# Start the visualizer.
# meshcat = StartMeshcat()

# def get_X_WG(self, context=None):
#     if not context:
#         context = self.CreateDefaultContext()
#     plant_context = self.plant.GetMyMutableContextFromRoot(context)
#     X_WG = self.plant.CalcRelativeTransform(
#                 plant_context,
#                 frame_A=self.world_frame,
#                 frame_B=self.gripper_frame)
#     return X_WG

def BuildAndSimulate():
    model_directives = """
    directives:
    - add_directives:
        file: package://manipulation/iiwa_and_wsg.dmd.yaml
    - add_model:
        name: floor
        file: package://manipulation/floor.sdf
    - add_weld:
        parent: world
        child: floor::box
        X_PC:
            translation: [0, 0, 0]
    """
    builder = DiagramBuilder()

    time_step = 0.001
    station = builder.AddSystem(
        MakeManipulationStation(model_directives, time_step=time_step))
    plant_robot = station.GetSubsystemByName("plant")
    iiwa = plant_robot.GetModelInstanceByName("iiwa")

    controller_plant = station.GetSubsystemByName(
        "iiwa_controller").get_multibody_plant_for_control()

    # Add a meshcat visualizer.
    meshcat.ResetRenderMode()
    meshcat.DeleteAddedControls()
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, station.GetOutputPort("query_object"), meshcat)

    # Set up differential inverse kinematics.
    differential_ik = AddIiwaDifferentialIK(
        builder,
        controller_plant,
        frame=controller_plant.GetFrameByName("iiwa_link_7"))
    builder.Connect(differential_ik.get_output_port(),
                    station.GetInputPort("iiwa_position"))
    builder.Connect(station.GetOutputPort("iiwa_state_estimated"),
                    differential_ik.GetInputPort("robot_state"))

    wsg_position = builder.AddSystem(ConstantVectorSource([0.1]))
    builder.Connect(wsg_position.get_output_port(),
                    station.GetInputPort("wsg_position"))

    diagram = builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    plant_context = plant_robot.GetMyContextFromRoot(context)
    station_context = station.GetMyContextFromRoot(context)
    diff_ik_context = differential_ik.GetMyContextFromRoot(context)

    # NN
    model = torch.nn.Linear(3,3)
    for t in range(10):
        # get gripper xyz position
        gripper_ind = plant_robot.GetBodyIndices(iiwa)[-1]
        gripper = plant_robot.get_body(gripper_ind)
        # gripper_init = gripper.EvalPoseInWorld(plant_context).translation()
        # # convert gripper xyz position from np to tensor
        # model_input = torch.Tensor(gripper_init) 
        # # get prediction
        # pred = model(model_input)
        # # pass prediction into diff ik to get joint angles
        # gripper_next = RigidTransform(RotationMatrix(), pred.detach().numpy())

        # test following trajectory
        gripper_next = test_inputs[t]
        print('gripper_next', gripper_next.translation())
        differential_ik.get_input_port(0).FixValue(
                    diff_ik_context, 
                    gripper_next)
        
        # display the joint angles
        print(
                "iiwa angles meas",
                station.GetOutputPort("iiwa_position_measured").Eval(station_context))
        # display the pose
        print(
                "iiwa pose meas",
                gripper.EvalPoseInWorld(plant_context).translation())
        # view the change in simulation
        simulator.AdvanceTo(t*time_step)

BuildAndSimulate()

gripper_next [ 4.68666315e-01 -1.06433466e-05  6.01285860e-01]
iiwa angles meas [-1.57  0.1   0.   -1.2   0.    1.6   0.  ]
iiwa pose meas [ 3.53636362e-04 -4.44084375e-01  7.66707814e-01]
gripper_next [0.55 0.   0.4 ]
iiwa angles meas [-1.57000000e+00  1.00000000e-01  3.98688060e-13 -1.20000000e+00
 -8.33064429e-14  1.60000000e+00  1.95755488e-13]
iiwa pose meas [ 3.53636362e-04 -4.44084375e-01  7.66707814e-01]
gripper_next [ 0.52660444 -0.06427876  0.4       ]
iiwa angles meas [-1.57000000e+00  1.00000000e-01  3.98688060e-13 -1.20000000e+00
 -8.33064429e-14  1.60000000e+00  1.95755488e-13]
iiwa pose meas [ 3.53636362e-04 -4.44084375e-01  7.66707814e-01]
gripper_next [ 0.46736482 -0.09848078  0.4       ]
iiwa angles meas [-1.56999986e+00  1.00000000e-01 -1.59730686e-07 -1.20000000e+00
  2.00213288e-08  1.60000000e+00 -1.39822158e-08]
iiwa pose meas [ 3.53636362e-04 -4.44084375e-01  7.66707814e-01]
gripper_next [ 0.4        -0.08660254  0.4       ]
iiwa angles meas [-1.56997144e+00  1.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a569df8b-a69d-4307-8cbd-4f24b608e3b4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>